数据来源 https://bosonnlp.com/dev/resource
命名实体识别

数据处理

In [1]:
import codecs
import os
from typing import Optional
import pandas as pd
import numpy as np
import pickle

In [2]:
def tagPhrase(sPath:str,tPath:str):
    '''
    phrase-based tag
    sPath: source file path
    tPath: target file path after tagged
    '''
    import jieba
    pass

In [128]:
sPath=r'F:\testDemo\AI\NER\ChineseNER-based-DL\dataset\BosonNLP_NER_6C\BosonNLP_NER_6C\BosonNLP_NER_6C.txt'
tPath=r'F:\testDemo\AI\NER\ChineseNER-based-DL\dataset\BosonNLP_NER_6C\BosonNLP_NER_6C\wordTagged.txt'
splitSenPattern='[，。《》“”‘’？！#【】]/O'
splitPath=r'F:\testDemo\AI\NER\ChineseNER-based-DL\dataset\BosonNLP_NER_6C\BosonNLP_NER_6C\split.txt'
sPreVector=r'F:\testDemo\AI\NER\ChineseNER-based-DL\dataset\sgns.renmin.word\sgns.renmin.word'
sentenceMaxLen=20
sModelFile=r'F:\testDemo\AI\NER\ChineseNER-based-DL\model\ner'
batch_size=64

In [4]:
def tagWords(sPath:str,tPath:str):
    '''
    word-based tag
    tag BIEO
    sPath: source file path
    tPath: target file path after tagged
    '''

    
    # del target file if existed
    if os.path.exists(tPath):
        os.remove(tPath)    
    
    with codecs.open(sPath,'r',encoding='utf-8') as r ,codecs.open(tPath,'w',encoding='utf-8') as w:
        for line in r: # process per line
#             print(line)
#             for i in range(len(line.rstrip())):
            i=0
            while i<len(line.rstrip()):
                if line[i]=='{' : # begin to tag entity
                    i+=2 # double { , valid entity
                    tmpEntity=[]
                    while line[i]!='}': #extract entity untill }
                        tmpEntity.append(line[i])
                        i+=1 # next word
                    ent=''.join(tmpEntity) # ent value is product_name:浙江在线杭州
                    word=ent.split(':') 
                    w.write('{}/B_{} '.format(word[1][0],word[0]))
                    for k in range(1,len(word[1])-1):
                        w.write('{}/I_{} '.format(word[1][k],word[0]))
                    w.write('{}/E_{} '.format(word[1][-1],word[0]))
                    i+=2 # next valid word  
                else:
                    w.write('{}/O '.format(line[i]))
                    i+=1
            w.write('\n')

In [5]:
tagWords(sPath=sPath,tPath=tPath)

In [6]:
def splitSentences(sPath: str,
                   splitPath: Optional[str] = None,
                   dicPath: Optional[str] = None):
    '''
    split into sentences from per text(the line of target file)
    
    sPath: file path needing to split
    tPath: optional, file path where all dics to be stored
    '''
    import re

    if not os.path.exists(sPath):
        raise Exception(' could not find the file path')
    allSentences = []  # record all sentence
    words = {'UNK': 0}  # k: word  v:frequency
    tags = {'UNK': 0}  # k: tag  v:frequency
    data = []
    labels = []
    if splitPath:
        w = codecs.open(splitPath, 'w', encoding='utf-8')
    with codecs.open(sPath, 'r', encoding='utf-8') as r:
        for line in r:
            sentences = re.split(
                splitSenPattern,
                line)  # using regular expression to split text into sentences
            for sen in sentences:  # sentences format :word/tag word/tag ...
                linedata = []
                linelabel = []
                sen=sen.strip()
                if sen is None or not sen:
                    continue
                if splitPath:
                    w.write('{}\n'.format(sen))
                for w_t in sen.split():  # w_t formatL word/tag
                    wt = w_t.split('/')
                    # record frequency for words and tags
                    linedata.append(wt[0])
                    linelabel.append(wt[1])
                    if wt[0] in words:
                        words[wt[0]] += 1
                    else:
                        words[wt[0]] = 1
                    if wt[1] in tags:
                        tags[wt[1]] += 1
                    else:
                        tags[wt[1]] = 1
                data.append(linedata)
                labels.append(linelabel)
    if splitPath:
        w.close()
    id2word = id2word_tag(words)
    id2tag = id2word_tag(tags)
    l = [id2word, id2tag, tag_word2id(id2word), tag_word2id(id2tag)]
    if dicPath:
        import pickle
        with open(dicPath, 'wb') as out:
            pickle.dump(l, out)
    return l, pd.DataFrame({'words': data, 'tags': labels})

In [7]:
def id2word_tag(dic:dict)->dict:
    '''
    format dict{id:word/tag}
    '''
    sorteddict=sorted(dic.items(),key=lambda x:x[1],reverse=True)
    import pandas as pd
    s=pd.Series(list(dict(sorteddict).keys()))
    return s.to_dict()


def tag_word2id(dic:dict)->dict:
    '''
    format dict{word/tag:id}
    
    dic: {id:word/tag}
    '''    
    return {v:k for k,v in dic.items()} 

In [8]:
l,df=splitSentences(sPath= tPath,splitPath=splitPath)

In [9]:
id2word,id2tag,word2id,tag2id=l

In [10]:
# word2id

In [11]:
# id2word

In [12]:
"id2word,id2tag,word2id,tag2id len is {},{},{},{} respectively ".format(len(id2word),len(id2tag),len(word2id),len(tag2id))

'id2word,id2tag,word2id,tag2id len is 3911,24,3911,24 respectively '

In [188]:
id2tag

{0: 'O',
 1: 'I_product_name',
 2: 'I_time',
 3: 'I_org_name',
 4: 'I_person_name',
 5: 'I_company_name',
 6: 'B_person_name',
 7: 'E_person_name',
 8: 'B_location',
 9: 'E_location',
 10: 'I_location',
 11: 'B_time',
 12: 'E_time',
 13: 'B_product_name',
 14: 'E_product_name',
 15: 'B_org_name',
 16: 'E_org_name',
 17: 'B_company_name',
 18: 'E_company_name',
 19: '',
 20: 'I_ime',
 21: 'B_ime',
 22: 'E_ime',
 23: 'UNK'}

In [13]:
# process train data
# df['tt']=len(df['words'])
df

,words,tags
0,"[浙, 江, 在, 线, 杭, 州, 4, 月, 2, 5, 日, 讯, （, 记, 者, ...","[B_product_name, I_product_name, I_product_nam..."
1,"[时, 髦]","[O, O]"
2,"[用, 微, 信, 交, 易, 毒, 品]","[O, B_product_name, E_product_name, O, O, O, O]"
3,"[没, 料, 想, 警, 方, 也, 很]","[O, O, O, O, O, O, O]"
4,[潮],[O]
5,"[将, 计, 就, 计]","[O, O, O, O]"
6,"[一, 举, 将, 其, 擒, 获]","[O, O, O, O, O, O]"
7,"[记, 者, 从, 杭, 州, 江, 干, 区, 公, 安, 分, 局, 了, 解, 到]","[O, O, O, B_org_name, I_org_name, I_org_name, ..."
8,"[经, 过, 一, 个, 多, 月, 的, 侦, 查, 工, 作]","[O, O, O, O, O, O, O, O, O, O, O]"
9,"[江, 干, 区, 禁, 毒, 专, 案, 组, 抓, 获, 吸, 贩, 毒, 人, 员, ...","[B_org_name, I_org_name, I_org_name, I_org_nam..."


In [170]:
df['wordslen']=df['words'].apply(lambda x:len(x) if len(x)<sentenceMaxLen else sentenceMaxLen)
print(df.describe()) # set sentence length is 20
print(df.info())

           wordslen
count  38314.000000
mean      11.502427
std        5.975219
min        1.000000
25%        6.000000
50%       11.000000
75%       17.000000
max       20.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38314 entries, 0 to 38313
Data columns (total 5 columns):
words       38314 non-null object
tags        38314 non-null object
wordslen    38314 non-null int64
x           38314 non-null object
y           38314 non-null object
dtypes: int64(1), object(4)
memory usage: 1.5+ MB
None


In [171]:
df.head(10)

,words,tags,wordslen,x,y
0,"[浙, 江, 在, 线, 杭, 州, 4, 月, 2, 5, 日, 讯, （, 记, 者, ...","[B_product_name, I_product_name, I_product_nam...",20,"[851, 391, 3, 310, 834, 343, 52, 45, 9, 35, 27...","[13, 1, 1, 1, 1, 14, 11, 2, 2, 2, 12, 0, 0, 0,..."
1,"[时, 髦]","[O, O]",2,"[26, 2873, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[用, 微, 信, 交, 易, 毒, 品]","[O, B_product_name, E_product_name, O, O, O, O]",7,"[43, 475, 165, 230, 488, 1136, 73, 0, 0, 0, 0,...","[0, 13, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
3,"[没, 料, 想, 警, 方, 也, 很]","[O, O, O, O, O, O, O]",7,"[181, 798, 299, 327, 62, 49, 180, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,[潮],[O],1,"[1159, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"[将, 计, 就, 计]","[O, O, O, O]",4,"[69, 196, 75, 196, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,"[一, 举, 将, 其, 擒, 获]","[O, O, O, O, O, O]",6,"[2, 557, 69, 95, 2723, 372, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,"[记, 者, 从, 杭, 州, 江, 干, 区, 公, 安, 分, 局, 了, 解, 到]","[O, O, O, B_org_name, I_org_name, I_org_name, ...",15,"[194, 70, 161, 834, 343, 391, 831, 176, 56, 20...","[0, 0, 0, 15, 3, 3, 3, 3, 3, 3, 3, 16, 0, 0, 0..."
8,"[经, 过, 一, 个, 多, 月, 的, 侦, 查, 工, 作]","[O, O, O, O, O, O, O, O, O, O, O]",11,"[50, 64, 2, 20, 44, 45, 0, 1342, 498, 94, 66, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,"[江, 干, 区, 禁, 毒, 专, 案, 组, 抓, 获, 吸, 贩, 毒, 人, 员, ...","[B_org_name, I_org_name, I_org_name, I_org_nam...",17,"[391, 831, 176, 1349, 1136, 357, 325, 454, 846...","[15, 3, 3, 3, 3, 3, 3, 16, 0, 0, 0, 0, 0, 0, 0..."


In [172]:
def getValuesBulk(dic: dict, keys: list) -> list:
    '''
    
    '''
    import pandas as pd
    s = pd.Series(dic, index=dic.keys())
    return list(s[keys])


def processData(df: pd.DataFrame, tag2id: dict, word2id: dict):
    def x_padding(words):
        wordids = []
        wordids = getValuesBulk(word2id, words)
        if len(wordids) < sentenceMaxLen:
            wordids.extend([0] * (sentenceMaxLen - len(wordids)))

        else:
            wordids = wordids[:sentenceMaxLen]
        return wordids
        # return wordids.extend([0] * (sentenceMaxLen - len(wordids))) if len(
        #     wordids) < sentenceMaxLen else wordids[:sentenceMaxLen]

    def y_padding(tags):
        tagids = []
        tagids = getValuesBulk(tag2id, tags)
        if len(tagids) < sentenceMaxLen:
            tagids.extend([0] * (sentenceMaxLen - len(tagids)))
        else:
            tagids = tagids[:sentenceMaxLen]
        return tagids

    df['x'] = df['words'].apply(x_padding)
    df['y'] = df['tags'].apply(y_padding)
    from sklearn.model_selection import train_test_split
    x_train, x_test = train_test_split(
        df.loc[:,:], test_size=0.3, random_state=40)

    # import pickle

    # with open('df', 'wb') as out:
    #     pickle.dump(df, out)

    return np.asarray(list(x_train['x'].values)), np.asarray(
        list(x_test['x'].values)), np.asarray(list(
            x_train['y'].values)), np.asarray(list(x_test['y'].values)),np.asarray(list(x_train['wordslen'])),np.asarray(list(x_test['wordslen']))

In [173]:
x_train, x_test, y_train, y_test,sequence_train,sequence_test=processData(df,tag2id,word2id)

In [58]:
def usingPreEmbedding(id2word:dict,spreVecPath:str,embedding_dim:int=300)->np.array:
    '''
    merge embedding vector using pre-trained vector
    
    word2id: current corpus word2id dict
    spreVecPath: the path of pre-trained embedding vector
    embedding_dim: embedding dim
    
    return: pre-trained vector
    '''
    import numpy as np
    # read pre-trained embedding vector
    pre={}
    with codecs.open(spreVecPath,'r',encoding='utf-8') as r:
        for line in r:
            row=line.rstrip().split()
            if len(row)==embedding_dim+1: # valid vector                
                pre[row[0]]=row[1:]
    
    # gen current embedding vector
    new_vector=[]
    for i in range(len(id2word)):
        word =id2word[i]
        if word in pre: # find an exists vector
#             print(type(pre[word]))
            new_vector.append(pre[word])
        else:
            z=np.random.random(size=embedding_dim)
#             print(type(z))
            new_vector.append(z)
            pass
    
    return np.asarray(new_vector,dtype=np.float32)

In [59]:
result=usingPreEmbedding(id2word=id2word,spreVecPath=sPreVector)

In [60]:
result.shape

(3911, 300)

In [61]:
word2id['UNK']
id2word[3910]

'UNK'

# generate batch

In [184]:
class BatchManager:
    def __init__(self,X,Y,sequence_length,shuffle=True):
        self.X=X
        self.Y=Y
        self.shuffle=shuffle
        self.start=0
        self.count=self.X.shape[0] # sample count
        self.sequence_length=sequence_length
        if self.shuffle:
            random_index=np.random.permutation(self.count)
            self.X=self.X[random_index]
            self.Y=self.Y[random_index]
            self.sequence_length=self.sequence_length[random_index]
    def next_batch(self,batch_size):
        end=self.start+batch_size
        b_X=self.X[self.start:end]
        b_Y=self.Y[self.start:end]
        sequence_length=self.sequence_length[self.start:end]
        if end >=self.count:
            self.start=0 # for next epoch
        else:
            self.start=end # for next batch
        return b_X,b_Y,sequence_length   

# build model

In [185]:
import tensorflow as tf

In [186]:
class Model:
    def __init__(self,voc_size:int=3911,lr:float=0.01,epoch:int=200,embedding_dim:int=300,usePreTrained:bool=False, \
                 pre_embedding=None,senMaxLen=20,tag_size=24,batch_size=64):
        self.lr=lr
        self.epoch=epoch
        self.voc_size=voc_size
        self.embedding_dim=embedding_dim
        self.pre_embedding=pre_embedding
        self.usePreTrained=usePreTrained
        self.senMaxLen=senMaxLen
        self.tag_size=tag_size
        self.batch_size=batch_size
        
        
    def buildNN(self):        
        self.inputdata=tf.placeholder(tf.int32,shape=[None,self.senMaxLen],name='input_data')
        self.tagdata=tf.placeholder(tf.int32,shape=[None,self.senMaxLen],name='label_data')
        self.sequence_length=tf.placeholder(tf.int32,shape=[None],name='real_length')
#         self.bsize=tf.placeholder(tf.int32,shape=(),name='real_batch_size')
        # get embedding voctor if embedding vector existed        
        with tf.variable_scope("embedding"):
            word_em=tf.get_variable('word_embedding',[self.voc_size,self.embedding_dim])
            if self.usePreTrained: # use pre-trained embedding vector
                word_em.assign(self.pre_embedding)
            lstm_input=tf.nn.embedding_lookup(word_em,self.inputdata)
        
        with tf.variable_scope('lstm'):
            fw=tf.contrib.rnn.LSTMBlockCell(self.embedding_dim,name='fw')
            bw=tf.contrib.rnn.LSTMBlockCell(self.embedding_dim,name='bw')
            # output shape is [batch_size,max_time,cell_output_size]
            (output_fw,output_bw),_=tf.nn.bidirectional_dynamic_rnn(fw,bw,lstm_input,dtype=tf.float32)
            lstm_output=tf.concat([output_fw,output_bw],axis=2)
            # new shape [x,self.embedding_dim*2] for calculating next z=xW+b
            lstm_output = tf.reshape(lstm_output, shape=[-1, self.embedding_dim*2])
        
        # full connect
        with tf.variable_scope('full_connect'):
            w=tf.get_variable('weight',[2*self.embedding_dim,self.tag_size],dtype=tf.float32)
            b=tf.get_variable('bias',[self.tag_size],dtype=tf.float32)
            dense_output=tf.nn.relu(tf.add(tf.linalg.matmul(lstm_output,w),b))
            # recover original shape for next crf
            dense_output=tf.reshape(dense_output, [-1, self.senMaxLen, self.tag_size])
            
        with tf.variable_scope('crf'):
            log,transition=tf.contrib.crf.crf_log_likelihood(dense_output,tag_indices=self.tagdata,sequence_lengths=self.sequence_length)
            self.decode_tags,self.best_score=tf.contrib.crf.crf_decode(dense_output,transition, self.sequence_length)
            
        self.loss=tf.math.reduce_mean(-log) 
        self.train_op=tf.train.AdamOptimizer(self.lr).minimize(self.loss)    

In [187]:
import math
tf.reset_default_graph()
batchManager=BatchManager(x_train,y_train,sequence_train)
model=Model()
model.buildNN()
with tf.Session() as sess:
    batch_num=math.ceil( x_train.shape[0]/model.batch_size)
    sess.run(tf.global_variables_initializer())
    saver=tf.train.Saver()
    step=1
    for epoch in range(200):
        for batch in range(batch_num):
            x,y,sequence_length=batchManager.next_batch(model.batch_size)
                        
            dic={model.inputdata:x,
                 model.tagdata:y ,
                 model.sequence_length:sequence_length
            }
            step+=1
            
            sqe,loss,_= sess.run([model.decode_tags,model.loss,model.train_op],dic)
            if step%10==0:
                print('step is {}, and loss is {}'.format(step,loss))
                print('sequence_length shape is {}'.format(sequence_length.shape))
#             print('b is {}'.format(b))
            if step%100==0:
                saver.save(sess,sModelFile,global_step=step) 

step is 10, and loss is 11.71469783782959
sequence_length shape is (64,)
step is 20, and loss is 8.949152946472168
sequence_length shape is (64,)
step is 30, and loss is 9.93435001373291
sequence_length shape is (64,)
step is 40, and loss is 7.988283157348633
sequence_length shape is (64,)
step is 50, and loss is 8.945716857910156
sequence_length shape is (64,)
step is 60, and loss is 8.610588073730469
sequence_length shape is (64,)
step is 70, and loss is 7.171231746673584
sequence_length shape is (64,)
step is 80, and loss is 6.097644805908203
sequence_length shape is (64,)
step is 90, and loss is 7.78274393081665
sequence_length shape is (64,)
step is 100, and loss is 6.35517692565918
sequence_length shape is (64,)
step is 110, and loss is 8.475770950317383
sequence_length shape is (64,)
step is 120, and loss is 5.614874362945557
sequence_length shape is (64,)
step is 130, and loss is 7.577733039855957
sequence_length shape is (64,)
step is 140, and loss is 4.260688781738281
sequenc

step is 1070, and loss is 2.341463327407837
sequence_length shape is (64,)
step is 1080, and loss is 1.8253087997436523
sequence_length shape is (64,)
step is 1090, and loss is 1.5358957052230835
sequence_length shape is (64,)
step is 1100, and loss is 1.3572955131530762
sequence_length shape is (64,)
step is 1110, and loss is 1.8110177516937256
sequence_length shape is (64,)
step is 1120, and loss is 1.6870620250701904
sequence_length shape is (64,)
step is 1130, and loss is 2.1248974800109863
sequence_length shape is (64,)
step is 1140, and loss is 1.1485695838928223
sequence_length shape is (64,)
step is 1150, and loss is 1.7239603996276855
sequence_length shape is (64,)
step is 1160, and loss is 2.1575241088867188
sequence_length shape is (64,)
step is 1170, and loss is 1.2930786609649658
sequence_length shape is (64,)
step is 1180, and loss is 1.3697597980499268
sequence_length shape is (64,)
step is 1190, and loss is 1.2481653690338135
sequence_length shape is (64,)
step is 1200,

step is 2160, and loss is 1.6335111856460571
sequence_length shape is (64,)
step is 2170, and loss is 1.646045446395874
sequence_length shape is (64,)
step is 2180, and loss is 1.3682148456573486
sequence_length shape is (64,)
step is 2190, and loss is 1.779736042022705
sequence_length shape is (64,)
step is 2200, and loss is 1.624283790588379
sequence_length shape is (64,)
step is 2210, and loss is 2.170806884765625
sequence_length shape is (64,)
step is 2220, and loss is 1.9746592044830322
sequence_length shape is (64,)
step is 2230, and loss is 2.460963249206543
sequence_length shape is (64,)
step is 2240, and loss is 0.981598436832428
sequence_length shape is (64,)
step is 2250, and loss is 1.4781663417816162
sequence_length shape is (64,)
step is 2260, and loss is 1.7042344808578491
sequence_length shape is (64,)
step is 2270, and loss is 1.8608535528182983
sequence_length shape is (64,)
step is 2280, and loss is 1.3985028266906738
sequence_length shape is (64,)
step is 2290, and 

step is 3250, and loss is 1.3824036121368408
sequence_length shape is (64,)
step is 3260, and loss is 1.9125068187713623
sequence_length shape is (64,)
step is 3270, and loss is 0.9540545344352722
sequence_length shape is (64,)
step is 3280, and loss is 1.1053335666656494
sequence_length shape is (64,)
step is 3290, and loss is 0.8559092879295349
sequence_length shape is (64,)
step is 3300, and loss is 1.5080171823501587
sequence_length shape is (64,)
step is 3310, and loss is 1.7419170141220093
sequence_length shape is (64,)
step is 3320, and loss is 1.800453543663025
sequence_length shape is (64,)
step is 3330, and loss is 1.0828887224197388
sequence_length shape is (64,)
step is 3340, and loss is 1.3778316974639893
sequence_length shape is (64,)
step is 3350, and loss is 1.522261619567871
sequence_length shape is (64,)
step is 3360, and loss is 1.5628430843353271
sequence_length shape is (64,)
step is 3370, and loss is 1.183276653289795
sequence_length shape is (64,)
step is 3380, a

step is 4340, and loss is 0.8827549815177917
sequence_length shape is (64,)
step is 4350, and loss is 1.2487304210662842
sequence_length shape is (64,)
step is 4360, and loss is 1.6081000566482544
sequence_length shape is (64,)
step is 4370, and loss is 1.4341709613800049
sequence_length shape is (64,)
step is 4380, and loss is 1.3323092460632324
sequence_length shape is (64,)
step is 4390, and loss is 1.337875247001648
sequence_length shape is (64,)
step is 4400, and loss is 1.4733080863952637
sequence_length shape is (64,)
step is 4410, and loss is 1.486114263534546
sequence_length shape is (64,)
step is 4420, and loss is 1.1307613849639893
sequence_length shape is (64,)
step is 4430, and loss is 1.7325819730758667
sequence_length shape is (64,)
step is 4440, and loss is 1.48081374168396
sequence_length shape is (64,)
step is 4450, and loss is 1.0325798988342285
sequence_length shape is (64,)
step is 4460, and loss is 0.9818010330200195
sequence_length shape is (64,)
step is 4470, an

...

step is 79340, and loss is 1.0565202236175537
sequence_length shape is (64,)
step is 79350, and loss is 0.5952547788619995
sequence_length shape is (64,)
step is 79360, and loss is 0.7751467823982239
sequence_length shape is (64,)
step is 79370, and loss is 0.7964887619018555
sequence_length shape is (64,)
step is 79380, and loss is 0.933042049407959
sequence_length shape is (64,)
step is 79390, and loss is 0.5946081876754761
sequence_length shape is (64,)
step is 79400, and loss is 0.9252773523330688
sequence_length shape is (64,)
step is 79410, and loss is 0.8000642657279968
sequence_length shape is (64,)
step is 79420, and loss is 1.187971830368042
sequence_length shape is (64,)
step is 79430, and loss is 0.5736342668533325
sequence_length shape is (64,)
step is 79440, and loss is 0.8359625339508057
sequence_length shape is (64,)
step is 79450, and loss is 0.9627601504325867
sequence_length shape is (64,)
step is 79460, and loss is 0.7331638932228088
sequence_length shape is (64,)
s

step is 80410, and loss is 0.8865056037902832
sequence_length shape is (64,)
step is 80420, and loss is 0.861831784248352
sequence_length shape is (64,)
step is 80430, and loss is 0.8776044845581055
sequence_length shape is (64,)
step is 80440, and loss is 0.6943701505661011
sequence_length shape is (64,)
step is 80450, and loss is 1.4058799743652344
sequence_length shape is (64,)
step is 80460, and loss is 0.9383630752563477
sequence_length shape is (64,)
step is 80470, and loss is 0.6416029334068298
sequence_length shape is (64,)
step is 80480, and loss is 0.5143316388130188
sequence_length shape is (64,)
step is 80490, and loss is 1.0097095966339111
sequence_length shape is (64,)
step is 80500, and loss is 0.7923834323883057
sequence_length shape is (64,)
step is 80510, and loss is 1.078946590423584
sequence_length shape is (64,)
step is 80520, and loss is 0.488341361284256
sequence_length shape is (64,)
step is 80530, and loss is 0.6871519088745117
sequence_length shape is (64,)
st

step is 81480, and loss is 0.6935123801231384
sequence_length shape is (64,)
step is 81490, and loss is 0.5199636816978455
sequence_length shape is (64,)
step is 81500, and loss is 0.6849164366722107
sequence_length shape is (64,)
step is 81510, and loss is 0.801219642162323
sequence_length shape is (64,)
step is 81520, and loss is 0.9415123462677002
sequence_length shape is (64,)
step is 81530, and loss is 0.782048225402832
sequence_length shape is (64,)
step is 81540, and loss is 0.9335031509399414
sequence_length shape is (64,)
step is 81550, and loss is 0.7677292823791504
sequence_length shape is (64,)
step is 81560, and loss is 0.7005554437637329
sequence_length shape is (64,)
step is 81570, and loss is 1.040074110031128
sequence_length shape is (64,)
step is 81580, and loss is 0.8795749545097351
sequence_length shape is (64,)
step is 81590, and loss is 1.2225046157836914
sequence_length shape is (64,)
step is 81600, and loss is 1.1747488975524902
sequence_length shape is (64,)
st

step is 82550, and loss is 1.1924540996551514
sequence_length shape is (64,)
step is 82560, and loss is 0.6865991353988647
sequence_length shape is (64,)
step is 82570, and loss is 0.5181549787521362
sequence_length shape is (64,)
step is 82580, and loss is 0.519928514957428
sequence_length shape is (64,)
step is 82590, and loss is 0.7912220358848572
sequence_length shape is (64,)
step is 82600, and loss is 1.0214017629623413
sequence_length shape is (64,)
step is 82610, and loss is 1.24045991897583
sequence_length shape is (64,)
step is 82620, and loss is 0.41108590364456177
sequence_length shape is (64,)
step is 82630, and loss is 0.6234664916992188
sequence_length shape is (64,)
step is 82640, and loss is 1.2213362455368042
sequence_length shape is (64,)
step is 82650, and loss is 0.777410626411438
sequence_length shape is (64,)
step is 82660, and loss is 0.5944721698760986
sequence_length shape is (64,)
step is 82670, and loss is 0.540974497795105
sequence_length shape is (64,)
ste

step is 83620, and loss is 0.993175745010376
sequence_length shape is (64,)
step is 83630, and loss is 0.7795412540435791
sequence_length shape is (64,)
step is 83640, and loss is 0.8836345672607422
sequence_length shape is (64,)
step is 83650, and loss is 0.7355375289916992
sequence_length shape is (64,)
step is 83660, and loss is 0.8053265810012817
sequence_length shape is (64,)
step is 83670, and loss is 1.017798900604248
sequence_length shape is (64,)
step is 83680, and loss is 0.8403549790382385
sequence_length shape is (64,)
step is 83690, and loss is 1.3635363578796387
sequence_length shape is (64,)
step is 83700, and loss is 1.1154863834381104
sequence_length shape is (64,)
step is 83710, and loss is 1.3770042657852173
sequence_length shape is (64,)
step is 83720, and loss is 0.6607691049575806
sequence_length shape is (64,)
step is 83730, and loss is 0.9517656564712524
sequence_length shape is (64,)
step is 83740, and loss is 0.8120877742767334
sequence_length shape is (64,)
s